In [38]:
import pandas as pd
import numpy as np

from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from tqdm import tqdm

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [4]:
train = pd.read_csv(r"train\train.csv")
test = pd.read_csv(r"test\test.csv")

In [5]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
lb= LabelEncoder()


In [6]:
X = train.text.values
y = lb.fit_transform(train.author.values)
y

array([2, 1, 0, ..., 0, 2, 2], dtype=int64)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)


In [49]:
embeddings_index = {}
f = open('glove.840B.300d.txt','r',encoding='utf-8')
for line in tqdm(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))



51644it [00:05, 8993.20it/s]


ValueError: could not convert string to float: '.'

The method isalpha() checks whether the string consists of alphabetic characters only.

In [40]:
lemm = WordNetLemmatizer()
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    words = [lemm.lemmatize(w) for w in words]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

In [41]:
xtrain_glove = [sent2vec(x) for x in tqdm(X_train)]
xvalid_glove = [sent2vec(x) for x in tqdm(X_test)]


 25%|██████████████████▉                                                         | 3898/15663 [00:04<00:14, 790.77it/s]Exception in thread Thread-19:
Traceback (most recent call last):
  File "C:\Users\ME\NEU\Anaconda\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\ME\NEU\Anaconda\lib\site-packages\tqdm\_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "C:\Users\ME\NEU\Anaconda\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

100%|████████████████████████████████████████████████████████████████████████████| 3916/3916 [00:01<00:00, 2163.66it/s]


In [42]:
xtrain_glove = np.array(xtrain_glove)
xvalid_glove = np.array(xvalid_glove)

In [43]:
# scale the data before any neural net:
scl = preprocessing.StandardScaler()
xtrain_glove_scl = scl.fit_transform(xtrain_glove)
xvalid_glove_scl = scl.transform(xvalid_glove)

In [44]:
ytrain_enc = np_utils.to_categorical(y_train)
yvalid_enc = np_utils.to_categorical(y_test)

In [50]:
# create a simple 3 layer sequential neural net
model = Sequential()

model.add(Dense(300, input_dim=300, activation='sigmoid'))
model.add(Dropout(0.2))


model.add(Dense(300, activation='sigmoid'))
model.add(Dropout(0.3))

model.add(Dense(3))
model.add(Activation('softmax'))

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [51]:
model.fit(xtrain_glove_scl, y=ytrain_enc, batch_size=64, 
          epochs=5, verbose=1, 
          validation_data=(xvalid_glove_scl, yvalid_enc))

Train on 15663 samples, validate on 3916 samples
Epoch 1/5
15663/15663 [==============================] - ETA: 4:38 - loss: 1.1585 - acc: 0.312 - ETA: 1:12 - loss: 1.1325 - acc: 0.394 - ETA: 33s - loss: 1.1274 - acc: 0.380 - ETA: 23s - loss: 1.1183 - acc: 0.39 - ETA: 18s - loss: 1.1233 - acc: 0.39 - ETA: 14s - loss: 1.1078 - acc: 0.42 - ETA: 12s - loss: 1.0881 - acc: 0.43 - ETA: 10s - loss: 1.0709 - acc: 0.45 - ETA: 9s - loss: 1.0678 - acc: 0.4566 - ETA: 8s - loss: 1.0523 - acc: 0.468 - ETA: 8s - loss: 1.0376 - acc: 0.481 - ETA: 7s - loss: 1.0160 - acc: 0.496 - ETA: 6s - loss: 1.0035 - acc: 0.507 - ETA: 6s - loss: 0.9992 - acc: 0.509 - ETA: 5s - loss: 0.9882 - acc: 0.519 - ETA: 5s - loss: 0.9803 - acc: 0.524 - ETA: 5s - loss: 0.9740 - acc: 0.527 - ETA: 5s - loss: 0.9728 - acc: 0.529 - ETA: 5s - loss: 0.9702 - acc: 0.530 - ETA: 5s - loss: 0.9665 - acc: 0.533 - ETA: 5s - loss: 0.9628 - acc: 0.535 - ETA: 5s - loss: 0.9634 - acc: 0.534 - ETA: 4s - loss: 0.9608 - acc: 0.536 - ETA: 4s - loss

In [20]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 70

token.fit_on_texts(list(X_train) + list(X_test))
xtrain_seq = token.texts_to_sequences(X_train)
xvalid_seq = token.texts_to_sequences(X_test)

# zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [47]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|████████████████████████████████████████████████████████████████████████| 25943/25943 [00:00<00:00, 345508.85it/s]


In [48]:
# A simple LSTM with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [29]:
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=2, verbose=1, validation_data=(xvalid_pad, yvalid_enc))

Train on 15663 samples, validate on 3916 samples
Epoch 1/2
15663/15663 [==============================] - ETA: 1:44 - loss: 1.1294 - acc: 0.335 - ETA: 1:16 - loss: 1.1239 - acc: 0.340 - ETA: 1:05 - loss: 1.1155 - acc: 0.360 - ETA: 58s - loss: 1.1178 - acc: 0.362 - ETA: 54s - loss: 1.1166 - acc: 0.37 - ETA: 51s - loss: 1.1155 - acc: 0.37 - ETA: 48s - loss: 1.1116 - acc: 0.37 - ETA: 45s - loss: 1.1103 - acc: 0.37 - ETA: 43s - loss: 1.1086 - acc: 0.38 - ETA: 41s - loss: 1.1077 - acc: 0.37 - ETA: 39s - loss: 1.1060 - acc: 0.37 - ETA: 37s - loss: 1.1058 - acc: 0.37 - ETA: 35s - loss: 1.1062 - acc: 0.37 - ETA: 33s - loss: 1.1047 - acc: 0.37 - ETA: 31s - loss: 1.1045 - acc: 0.37 - ETA: 29s - loss: 1.1035 - acc: 0.37 - ETA: 27s - loss: 1.1030 - acc: 0.37 - ETA: 25s - loss: 1.1015 - acc: 0.37 - ETA: 23s - loss: 1.1003 - acc: 0.38 - ETA: 21s - loss: 1.0999 - acc: 0.38 - ETA: 20s - loss: 1.0988 - acc: 0.38 - ETA: 17s - loss: 1.0975 - acc: 0.38 - ETA: 15s - loss: 1.0964 - acc: 0.38 - ETA: 13s - lo

In [ ]:
#https://github.com/nikbearbrown/NEU_COE/blob/master/Deep_Learning/RNNs/7.1%20RNN%20and%20LSTM.ipynb
seq = Sequential()
seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   input_shape=(None, 40, 40, 1),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(Conv3D(filters=1, kernel_size=(3, 3, 3),
               activation='sigmoid',
               padding='same', data_format='channels_last'))
seq.compile(loss='binary_crossentropy', optimizer='adadelta')